In [62]:
class TobiiObject:
    
    #Basic accesable variables
    data = []
    headers = []
    participants = {}
    
    #Init
    def __init__(self, tsvPath):
        self.addParticipants(tsvPath)
        
    #Init method
    def _importTobiiTsv(self, tsvPath):
        #Import csv library
        import csv
        #Create a container for temporary data
        tempData = []
        tempHeaders = []
        index = 0
        #Open the .tsv file and append it to a python list
        with open(tsvPath,'rb') as tsvIn:
            tsvIn = csv.reader(tsvIn, delimiter='\t')
            for row in tsvIn:
                tempData.append(row)
        #Extract all the headers and append to a python list with the index
        for header in tempData[0]:
            tempHeaders.append(str(header))
            #print str(index) + ": " + header
            index += 1
        #Remove the headers from the data list
        tempData = tempData[1:]
        #Return the two data lists
        return [tempData, tempHeaders]
    
    # ---------- Public methods! ---------------
    def addParticipants(self, tsvPath):
        #Load the data and headers
        data, self.headers = self._importTobiiTsv(tsvPath)
        #Load the participants
        ###  self.participants = self._getParticipants()
        #Load the individual participants mediarows
        ###  self.participants = self._getMediaRowForParticipants(self._getParticipants())
        participants = self._getParticipants(data)
        self.addMediaRowForParticipants(self._getMediaRowForParticipants(participants, data))
        #Create saccade data for each participant for each media element [AmountOfSaccades, SaccadeMeanDuratio, SaccadeMeanVelocity]
        self._createSaccadeData(data, participants)
        #Create pupil dilation data for each participant for each media element [Mean pupil dilation, mean pupil variance, mean left pupil dilation, mean right pupil dilation]
        self._createPupilData(data, participants)
        
    def printHeaders(self):
        index = 0
        for header in self.headers:
            print str(index) + ": " + header
            index += 1
        
    # !!!!!!!!! PRIVATE METHODS !!!!!!!!!!!!
    
    def _getHeaderIndex(self, headerName):
        #Getting header index because header move according to content of the Tobii test (questions, AOI's, etc..)
        index = 0
        #Search through the headers and return if the header name is found
        for header in self.headers:
            if header == headerName:
                return index
            index += 1
            #If not found return "ERROR"
        return "ERROR"
    
    def _getParticipants(self, data):
        tempParticipants = {}
        #Get the index for ParticipantName
        participantNameIndex = self._getHeaderIndex("ParticipantName")
        #Go through the data and add participant names that are not already in the dictionary
        for row in data:
            if row[participantNameIndex] not in tempParticipants.keys():
                tempParticipants[row[participantNameIndex]] = {}
        return tempParticipants
        
    def _getMediaRowForParticipants(self, participantsIn, data):
        #Get indexes for MediaName and ParticipantName
        participantNameIndex = self._getHeaderIndex("ParticipantName")
        mediaNameIndex = self._getHeaderIndex("MediaName")
        #For each participant get the datarows for each media element, then afterwards get the starting row and ending row
        for participant in participantsIn:
            media = {}
            index = 0
            for row in data:
                if participant == row[participantNameIndex] and row[mediaNameIndex] != "":
                    if row[mediaNameIndex] not in media:
                        media[row[mediaNameIndex]] = [index]
                    else:
                        media[row[mediaNameIndex]].append(index)
                index += 1
            media2 = {}
            for key in media.keys():
                first = media[key][0]
                last = media[key][-1]
                media2[key] = [first, last]
            participantsIn[participant]["media"] = media2
        return participantsIn
    
    def _stringToFloat(self, string):
        for idx in xrange(len(string)):
            if string[idx] == ".":
                return float(string[:idx]) + float(string[idx:])
        return "ERROR"
    
    def addMediaRowForParticipants(self, participantsIn):
        for participant in participantsIn:
            self.participants[participant] = {}
            self.participants[participant]["media"] = participantsIn[participant]["media"]
    
    #------------- SACCADE METHODS-----------------------         
    def _calculateSaccadeAmount(self, saccadeList):
            return len(saccadeList)
    
    def _calculateSaccadeDurationMean(self, durationDict):
        dictSum = sum(durationDict.values())
        dictLen = len(durationDict.values())
        return float(dictSum) / float(dictLen)
    
    def _calculateSaccadeVelocityMean(self, velocityDict):
        velSum = sum(velocityDict.values())
        velLen = len(velocityDict.values())
        return float(velSum) / float(velLen)
            
    def _createSaccadeData(self, data, participants):
        #Saccade index in the headers
        sacIdx = self._getHeaderIndex("SaccadeIndex")
        recTimestampIdx = self._getHeaderIndex("RecordingTimestamp")
        sacAmpIdx = self._getHeaderIndex("SaccadicAmplitude")
        #For each participant
        for participant in participants:
            self.participants[participant]["saccadeData"] = {}
            #Fir each media element
            for image in self.participants[participant]["media"]:
                #Set the starting and ending data row indexes
                fromRow, toRow = self.participants[participant]["media"][image]
                #Variables
                #Index to hold all of the indexes in order to count the amount of saccades
                index = []
                #Duration dictionary to hold the duration of each saccade in the index
                duration = {}
                #Velocity dictionary in order to hold the velocity for each saccade in the index
                velocity = {}
                #Previous and current saccade indexes to compare eachother
                prevSacIdx = None
                currSacIdx = None
                #Saccade duration start and end in order to calculate the duration
                sacDurStart = 0
                sacDurEnd = 0
                for row in data[fromRow:toRow]:
                    #Update the previous and current saccade indexes
                    prevSacIdx = currSacIdx
                    currSacIdx = row[sacIdx]
                    # If the saccade index is not already in the list, append it
                    if row[sacIdx] not in index and row[sacIdx] != "":
                        index.append(row[sacIdx])
                    #Check if previous was empty and current is a saccadeIndex then set the start time
                    if currSacIdx != "" and currSacIdx != None and prevSacIdx == "":
                        sacDurStart = row[recTimestampIdx]
                    #Check if previous is a saccade and current is not, then set the end time
                    elif prevSacIdx != "" and currSacIdx == "":
                        sacDurEnd = row[recTimestampIdx]
                        #Calculate the duration of the saccade
                        duration[prevSacIdx] = int(sacDurEnd) - int(sacDurStart)
                        #Cant remember it right now will write later
                        if row[sacAmpIdx] != "":
                            curAmp = str(row[sacAmpIdx]).replace(",",".")
                            curAmp = self._stringToFloat(curAmp)
                            #Get the duration and converto to seconds
                            curDur = float(duration[prevSacIdx])/1000 #Seconds
                            #Calculate the velocity
                            velocity[prevSacIdx] = curAmp / curDur
                            
                indexTotal = self._calculateSaccadeAmount(index)
                durationMean = self._calculateSaccadeDurationMean(duration)
                velocityMean = self._calculateSaccadeVelocityMean(velocity)
                self.participants[participant]["saccadeData"][image] = [indexTotal, durationMean, velocityMean]
                
    # ------------------- PUPIL DILATION ------------------------------
    def _createPupilData(self, data, participants):
        #Import numpy library
        import numpy as np
        #Index in the headers
        pupilLeftIdx = self._getHeaderIndex("PupilLeft")
        pupilRightIdx = self._getHeaderIndex("PupilRight")
        #For each participant
        for participant in participants:
            self.participants[participant]["pupilData"] = {}
            #Fir each media element
            for image in self.participants[participant]["media"]:
                #Set the starting and ending data row indexes
                fromRow, toRow = self.participants[participant]["media"][image]
                #Variables to hold the data
                pupilListLeft = []
                pupilListRight = []
                pupilListMean = []
                #Go through the data and create a list with the pupil data
                for row in data[fromRow:toRow]:
                    if row[pupilLeftIdx] != "" and row[pupilRightIdx] != "":
                        tmpPupilLeft = str(row[pupilLeftIdx]).replace(",",".")
                        tmpPupilLeft = self._stringToFloat(tmpPupilLeft)
                        pupilListLeft.append(tmpPupilLeft)
                        tmpPupilRight = str(row[pupilRightIdx]).replace(",",".")
                        tmpPupilRight = self._stringToFloat(tmpPupilRight)
                        pupilListRight.append(tmpPupilRight)
                        tmpPupilMean = (tmpPupilLeft + tmpPupilRight) / 2
                        pupilListMean.append(tmpPupilMean)
                #Create mean values for that list
                pupilLeftMean = sum(pupilListLeft)/len(pupilListLeft)
                pupilRightMean = sum(pupilListRight)/len(pupilListRight)
                pupilMeanMean = sum(pupilListMean)/len(pupilListMean)
                pupilVariance = np.var(pupilListMean)
                self.participants[participant]["pupilData"][image] = [pupilMeanMean, pupilVariance, pupilLeftMean, pupilRightMean]
                  

In [63]:
tobii = TobiiObject("pilot4participants.tsv")

In [64]:
print tobii.participants.keys()

['P02B', 'P04A', 'P01A', 'P03B']


In [65]:
tobii.addParticipants("pilot1participant.tsv")

In [68]:
print tobii.participants["P05A"]["saccadeData"]

{'M4A-Q24.png': [30, 10051.645161290322, 182.870769744843], 'M4A-Q21.png': [22, 12196.04347826087, 204.9490432104295], 'M4A-Q22.png': [26, 10744.185185185184, 149.62583098682936], 'M4A-Q23.png': [32, 9117.727272727272, 220.85325854245045], 'M3A-Q18.png': [65, 3696.757575757576, 174.82202473747716], 'M2A-Q09.png': [10, 13807.1, 158.37206884869477], 'M2A-Q12.png': [36, 4484.918918918919, 136.51572243873306], 'M2A-Q10.png': [18, 7676.684210526316, 120.23875863168772], 'M4A-Q20.png': [29, 9030.366666666667, 160.9129301836566], 'TUTX-Q1.png': [18, 262.57894736842104, 143.57694287658646], 'TUTX-Q2.png': [14, 814.2, 139.48447317820575], 'M1A-Q04.png': [31, 2547.5666666666666, 239.0097294221315], 'M1A-Q06.png': [33, 2981.794117647059, 175.2441941722772], 'M2A-Q08.png': [28, 4341.0344827586205, 124.59486733548079], 'M4A-Q19.png': [25, 10030.884615384615, 131.80109446662703], 'M2A-Q11.png': [25, 5976.2307692307695, 234.7901288282923], 'M3A-Q13.png': [31, 5560.625, 129.1473169594376], 'M1A-Q01.pn

In [67]:
print tobii.printHeaders()

0: ﻿ExportDate
1: StudioVersionRec
2: StudioProjectName
3: StudioTestName
4: ParticipantName
5: [M1BE-Q1]Value
6: [M1BE-Q2]Value
7: [M1BE-Q3]Value
8: [M1BE-Q4]Value
9: [M1BE-Q5]Value
10: [M1BE-Q6]Value
11: [M1B-Q01]Value
12: [M1B-Q02]Value
13: [M1B-Q03]Value
14: [M1B-Q04]Value
15: [M1B-Q05]Value
16: [M1B-Q06]Value
17: [M2BE-Q10]Value
18: [M2BE-Q11]Value
19: [M2BE-Q12]Value
20: [M2BE-Q7]Value
21: [M2BE-Q8]Value
22: [M2BE-Q9]Value
23: [M2B-Q07]Value
24: [M2B-Q08]Value
25: [M2B-Q09]Value
26: [M2B-Q10]Value
27: [M2B-Q11]Value
28: [M2B-Q12]Value
29: [M3BE-Q13]Value
30: [M3BE-Q14]Value
31: [M3BE-Q15]Value
32: [M3BE-Q16]Value
33: [M3BE-Q17]Value
34: [M3BE-Q18]Value
35: [M3B-Q13]Value
36: [M3B-Q14]Value
37: [M3B-Q15]Value
38: [M3B-Q16]Value
39: [M3B-Q17]Value
40: [M3B-Q18]Value
41: [M4BE-Q19]Value
42: [M4BE-Q20]Value
43: [M4BE-Q21]Value
44: [M4BE-Q22]Value
45: [M4BE-Q23]Value
46: [M4BE-Q24]Value
47: [M4B-Q19]Value
48: [M4B-Q20]Value
49: [M4B-Q21]Value
50: [M4B-Q22]Value
51: [M4B-Q23]Value
52: 